**Завантажую бібліотеку surprise**

In [1]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357249 sha256=11fba0a949f08825faeb37d512c41f3cfcee610b86607017a76b2c08ed2389ed
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


**Завантажую датасет та отримую інформацію про нього**

In [3]:
# Завантажую потрібні бібліотеки
import pandas as pd
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD,SVDpp,NMF
from collections import defaultdict
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise import accuracy

In [4]:
# Завантажую датасет
data=Dataset.load_builtin('ml-100k')

reader=Reader(rating_scale=(1,5))

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [5]:
# Отримання перших 5 рядків датасету
first_five_rows = data.raw_ratings[:5]
# Виведення перших 5 рядків
print("Перші 5 рядків датасету:")
for row in first_five_rows:
    print(row)

Перші 5 рядків датасету:
('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


In [6]:
# Отримання кількості рейтингів у датасеті
num_ratings = len(data.raw_ratings)
print("Кількість рейтингів у датасеті:", num_ratings)


Кількість рейтингів у датасеті: 100000


In [7]:
# З метою отримання інформації про датасет намагаюсь вивести кількість унікальних користувачів та об'єктів
unique_users = len(set([user for user, _, _, _ in data.raw_ratings]))
unique_items = len(set([item for _, item, _, _ in data.raw_ratings]))
print("Кількість унікальних користувачів:", unique_users)
print("Кількість унікальних об'єктів (фільмів):", unique_items)


Кількість унікальних користувачів: 943
Кількість унікальних об'єктів (фільмів): 1682


**Використання алгоритму GridSearchCV для отримання найкращих параметрів моделі**

In [8]:
# Для використання алгоритму GridSearchCV розділяю дані на навчальну та тестову вибірки
trainset, testset = train_test_split(data, test_size=0.3)

In [9]:
trainset

In [10]:
testset[:5]

[('892', '215', 4.0),
 ('730', '237', 3.0),
 ('148', '588', 4.0),
 ('655', '212', 3.0),
 ('307', '1028', 4.0)]

In [11]:
# Визначаю параметри для грід пошуку як такі, що вважаються класичними для використання
param_grid = {
    'n_factors': [20, 50, 100],
    'n_epochs': [20, 30, 40],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.1, 0.4]
}
param_grid

{'n_factors': [20, 50, 100],
 'n_epochs': [20, 30, 40],
 'lr_all': [0.002, 0.005, 0.01],
 'reg_all': [0.02, 0.1, 0.4]}

In [12]:
# Оскільки алгоритм GridSearchCV не працює коректно із trainset для подальшої роботи із trainset перетворюю trainset у список кортежів (user_id, item_id, rating)
trainset_tuples = [(uid, iid, rating) for (uid, iid, rating) in trainset.all_ratings()]

# Збережую списку кортежів у файл
with open('trainset_file.txt', 'w') as f:
    for uid, iid, rating in trainset_tuples:
        f.write(f"{uid}, {iid}, {rating}\n")

# Трансформую навчальний датасет з файлу на об'єкт Dataset
data_from_file = Dataset.load_from_file('trainset_file.txt', reader=reader)

# З метою підбору найкращих параметрів за допомогою крос-валідації ініціалізую GridSearchCV
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5, n_jobs=-1, pre_dispatch='2*n_jobs')

# Запуск GridSearchCV на навчальному датасеті
grid_search.fit(data_from_file)

In [13]:
# Вивожу найкращі параметри
best_params = grid_search.best_params
print("Найкращі параметри:", best_params)

# Отримую найкращі результати
best_score = grid_search.best_score
print("Найкращий результат:", best_score)

Найкращі параметри: {'rmse': {'n_factors': 100, 'n_epochs': 40, 'lr_all': 0.01, 'reg_all': 0.1}, 'mae': {'n_factors': 50, 'n_epochs': 40, 'lr_all': 0.01, 'reg_all': 0.1}}
Найкращий результат: {'rmse': 0.9297152546416074, 'mae': 0.7348733031206288}


**Навчання моделі за алгоритмом SVD без кросвалідації і прогнозування рейтингів з використанням найкращих параметрів**

In [15]:
# Ініціалізую алгоритм SVD з найкращими параметрами
best_algo = SVD(n_factors=100, n_epochs=40, lr_all=0.01, reg_all=0.1)

# Навчаю модель на всій навчальній вибірці
best_algo.fit(trainset)

**Прогнозування рейтингів, отриманих для навчальної виборки на тестовій вибірці**

In [36]:
# Прогнозування рейтингів на тестовій вибірці
predictions = best_algo.test(testset)

# Оцінка ефективності моделі
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)
fcp = accuracy.fcp(predictions)

RMSE: 0.9198
MAE:  0.7283
FCP:  0.7234


**Проведення кросвалідації за алгоритмом SVD у навчальному датасеті**

In [25]:
# Крос-валідація алгоритму SVD
results = cross_validate(best_algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Виведення середніх значень RMSE та MAE
print("Середнє RMSE:", results['test_rmse'].mean())
print("Середнє MAE:", results['test_mae'].mean())

# Навчання моделі на всій навчальній вибірці
best_algo.fit(trainset)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9057  0.9184  0.9021  0.9175  0.9074  0.9102  0.0065  
MAE (testset)     0.7142  0.7245  0.7134  0.7285  0.7164  0.7194  0.0060  
Fit time          3.42    3.00    3.33    3.68    3.03    3.29    0.25    
Test time         0.35    0.13    0.43    0.14    0.13    0.24    0.13    
Середнє RMSE: 0.9102132946196587
Середнє MAE: 0.7193964281034717


In [26]:
# Для порівняння той самий код, але без врахування параметрів моделі, отриманих через GridSearch. Результат гірший, але не небагато. Тому надалі в коді ці параметри використовую
algo = SVD()

results_1= cross_validate(algo, data_from_file, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9532  0.9492  0.9538  0.9501  0.9464  0.9505  0.0027  
MAE (testset)     0.7529  0.7503  0.7521  0.7525  0.7487  0.7513  0.0016  
Fit time          1.17    1.52    0.96    0.94    0.92    1.10    0.23    
Test time         0.34    0.16    0.09    0.09    0.24    0.18    0.09    


In [28]:
algo_svdpp = SVDpp(n_factors=100, n_epochs=40, lr_all=0.01, reg_all=0.1)

# Run 5-fold cross-validation and print results.
results_svdpp= cross_validate(algo_svdpp, data_from_file, measures=['RMSE', 'MAE'], cv=5, verbose=True)
# Виведення середніх значень RMSE та MAE


Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9245  0.9250  0.9284  0.9278  0.9191  0.9250  0.0033  
MAE (testset)     0.7314  0.7303  0.7321  0.7351  0.7271  0.7312  0.0026  
Fit time          120.39  118.92  120.40  118.36  120.24  119.66  0.86    
Test time         2.45    2.63    2.56    2.58    2.50    2.54    0.06    


In [37]:
# Навчання моделi SVD++  на всій навчальній вибірці
algo_svdpp.fit(trainset)

In [39]:
# Прогнозування рейтингів, отриманих на навчальній виборці за алгоритмом SVD++ на тестовій вибірці

# Прогнозування рейтингів на тестовій вибірці
predictions_svdpp = algo_svdpp.test(testset)

# Оцінка ефективності моделі
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)
fcp = accuracy.fcp(predictions)

RMSE: 0.9198
MAE:  0.7283
FCP:  0.7234


In [32]:
algo_nmf = NMF(n_factors=100, n_epochs=40, reg_pu=0.1, reg_qi=0.1)

# Крос-валідація алгоритму NMF
results_nmf = cross_validate(algo_nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9606  0.9686  0.9727  0.9735  0.9838  0.9718  0.0075  
MAE (testset)     0.7416  0.7471  0.7482  0.7495  0.7565  0.7486  0.0048  
Fit time          4.42    4.39    5.42    5.32    5.72    5.05    0.55    
Test time         0.11    0.12    0.12    0.14    0.12    0.12    0.01    


In [40]:
# Навчання моделі NMF на всій навчальній вибірці
algo_nmf.fit(trainset)

In [41]:
# Прогнозування рейтингів, отриманих на навчальній виборці за алгоритмом NMF на тестовій вибірці

# Прогнозування рейтингів на тестовій вибірці
predictions_nmf = algo_nmf.test(testset)

# Оцінка ефективності моделі
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)
fcp = accuracy.fcp(predictions)

RMSE: 0.9198
MAE:  0.7283
FCP:  0.7234


In [52]:
# Вивожу в результуючій таблиці значення метрик якості моделей, навчених і кросвалідованих різними алгоритмами за результатами навчальної вибірки

# Значення RMSE та MAE для різних алгоритмів
results = {
    'Algo': ['SVD', 'SVD++', 'NMF', 'TESTSET'],
    'RMSE MEAN': [0.9102, 0.9250, 0.9718,  0.9198],
    'RMSE STD': [0.0065, 0.0033, 0.0075, '-'],
    'MAE MEAN': [0.7194, 0.7312, 0.7486, 0.7283],
    'MAE STD': [0.0060, 0.0026, 0.0048, '-']
}

# Створення DataFrame зі словника
df = pd.DataFrame(results)

# Виведення оновленої таблиці
print(df)


      Algo  RMSE MEAN RMSE STD  MAE MEAN MAE STD
0      SVD     0.9102   0.0065    0.7194   0.006
1    SVD++     0.9250   0.0033    0.7312  0.0026
2      NMF     0.9718   0.0075    0.7486  0.0048
3  TESTSET     0.9198        -    0.7283       -


**ВИСНОВОК 1.** Виходячи з отриманих результатів, можна зробити висновок, що кращий результат показувало навчання з використанням параметрів алгоритму GridSearchCV. Виконання кросвалідації дало найкращі результати за алгоритмом SVD++, оскільки при незначному переважання середнього значення метрик якості був вдвічі нижче розкид даних. РЕзультати тестової вибірки також були найближчими до результатів за алгоритмом SVD++.

In [42]:
def get_top_n(predictions,n=4):
  top_n=defaultdict(list)
  for uid,iid,true_r,est,_ in predictions:
    top_n[uid].append((iid,est))

  for uid, user_rating in top_n.items():
    user_rating.sort(key=lambda x: x[1], reverse=True)
    top_n[uid]=user_rating[:n]
  return top_n

In [44]:
top_n=get_top_n(predictions,n=4)

In [46]:
# Виведення рекомендацій для перших 10 споживачів
print("Рекомендації для перших 10 споживачів:")
for i, (uid, user_ratings) in enumerate(top_n.items()):
    if i >= 10:
        break
    print(f"Користувач {uid}: {[iid for (iid, _) in user_ratings]}")

Рекомендації для перших 10 споживачів:
Користувач 892: ['12', '183', '479', '127']
Користувач 730: ['50', '1', '237', '100']
Користувач 148: ['135', '357', '177', '98']
Користувач 655: ['357', '178', '251', '657']
Користувач 307: ['114', '169', '50', '173']
Користувач 932: ['661', '134', '98', '199']
Користувач 890: ['134', '515', '603', '480']
Користувач 447: ['50', '89', '209', '223']
Користувач 940: ['174', '313', '194', '427']
Користувач 160: ['182', '127', '285', '135']


In [47]:
# Отримання топ-4 рекомендацій для кожного користувача при використанні алгоритму NMF
top_n = get_top_n(predictions_nmf, n=4)

# Виведення рекомендаційдля 10 перших користувачіва
print("Рекомендації для перших 10 споживачів:")
for i, (uid, user_ratings) in enumerate(top_n.items()):
    if i >= 10:
        break
    print(f"Користувач {uid}: {[iid for (iid, _) in user_ratings]}")

Рекомендації для перших 10 споживачів:
Користувач 892: ['191', '168', '12', '479']
Користувач 730: ['50', '100', '1', '340']
Користувач 148: ['357', '174', '127', '98']
Користувач 655: ['12', '515', '1554', '357']
Користувач 307: ['169', '114', '50', '178']
Користувач 932: ['64', '134', '515', '199']
Користувач 890: ['515', '603', '480', '134']
Користувач 447: ['50', '89', '223', '209']
Користувач 940: ['174', '285', '427', '709']
Користувач 160: ['127', '59', '285', '50']


In [48]:
# Отримання топ-4 рекомендацій для кожного користувача при використанні алгоритму SVD++
top_n = get_top_n(predictions_svdpp, n=4)

# Виведення рекомендацій для 10 перших користувачів
print("Рекомендації для перших 10 споживачів:")
for i, (uid, user_ratings) in enumerate(top_n.items()):
    if i >= 10:
        break
    print(f"Користувач {uid}: {[iid for (iid, _) in user_ratings]}")

Рекомендації для перших 10 споживачів:
Користувач 892: ['12', '641', '127', '172']
Користувач 730: ['50', '100', '237', '1']
Користувач 148: ['357', '190', '127', '135']
Користувач 655: ['657', '178', '1169', '515']
Користувач 307: ['169', '114', '173', '178']
Користувач 932: ['661', '100', '199', '185']
Користувач 890: ['530', '134', '480', '515']
Користувач 447: ['50', '89', '79', '211']
Користувач 940: ['313', '174', '285', '430']
Користувач 160: ['127', '182', '285', '135']


**ВИСНОВОК 2.** Вивід рекомендованих фільмів для користувачів за різними алгоритмами показава деякі неспівпадіння у сетах пропонованих фільмів. Беручи до уваги попередні результати (висновок 1), я би пропонувала набір фільмів за алгоритмом SVD++.